In [ ]:
from granite_io import make_backend
from granite_io.io import make_io_processor
from granite_io.io.base import ChatCompletionInputs
from granite_io.io.voting import MBRDMajorityVotingProcessor

In [ ]:
%pip install openai

# Connect to a backend running on localhost.
port = 11434
url = f"http://localhost:{port}/v1"
model_name = "granite3.3:8b"
backend = make_backend(
    "openai",
    {
        "model_name": model_name,
        "openai_base_url": url,
    },
)

In [ ]:
# Ask a question with an integral answer
# question = "If 4 daps = 7 yaps, and 5 yaps = 3 baps, how many daps equal 42 baps?" # gt: 40
# question = "How many ways are there to put 5 balls in 2 boxes if the balls are not distinguishable and neither are the boxes?" # gt: 3
question = "Paul and Jesse each choose a number at random from the first six primes. What is the probability that the sum of the numbers they choose is even?"  # gt: \frac{13}{18}
completion_inputs = ChatCompletionInputs(
    messages=[
        {
            "role": "user",
            "content": f"{question}\nPlease reason step by step, and put your final answer within \\boxed{{}}",
        }
    ],
    thinking=True,
    generate_inputs={"n": 4, "temperature": 1.0, "max_tokens": 2048},
)
completion_inputs

In [ ]:
# Run the question through the base model
base_processor = make_io_processor("Granite 3.3", backend=backend)
results = base_processor.create_chat_completion(completion_inputs)
print("*** Output from base model ***\n")
print(results.results[0].next_message.content)

In [ ]:
# Wrap the base model's I/O processor in an MBRD I/O processor.
voting_processor = MBRDMajorityVotingProcessor(base_processor)
results = voting_processor.create_chat_completion(completion_inputs)
print("*** Top 1 output from base model selected using MBRD ***\n")
print(results.results[0].next_message.content)
# What's the actual answer?
print(f"---------\nThe actual answer is: \\frac{{13}}{{18}}")